In [ ]:
# Notebook for debugging the pipline

In [2]:
from pipeline import Pipeline

json_path = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/23/data_sets/nyc_squad_with_pretraining_commands.jsonl"

pipeline = Pipeline(prepared_dataset_filepath=json_path)
pipeline.load_prepared_dataset()
#pipeline.run_pre_training_evaluation_testing()
#pipeline.run_evaluate_pre_training_results()
#pipeline.generate_pre_training_data()


2026-01-30 13:24:23,899 - pipeline - INFO - Prepared dataset loaded successfully.


In [2]:
pipeline.run_evaluate_pre_training_results()

2026-01-30 08:26:24,361 - pipeline - INFO - Starting evaluation of pre-training results...
2026-01-30 08:26:24,371 - testing - INFO - length of log file questions and answers: 129
2026-01-30 08:26:24,410 - testing - INFO - number of test samples not found in training data: 0
2026-01-30 08:26:24,413 - testing - INFO - test samples not found in training data: [] ...


Generating test answer embeddings:   0%|          | 0/129 [00:00<?, ?it/s]

2026-01-30 08:26:24,947 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Generating test answer embeddings:   2%|▏         | 2/129 [00:00<00:32,  3.90it/s]

2026-01-30 08:26:25,057 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


...

2026-01-30 08:26:53,527 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Generating response answer embeddings: 100%|██████████| 129/129 [00:14<00:00,  8.87it/s]

2026-01-30 08:26:53,634 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Generating response answer embeddings: 100%|██████████| 129/129 [00:14<00:00,  8.87it/s]


2026-01-30 08:26:53,780 - testing - INFO - test results written to /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/23/testing/squad_dataset_with_pre_training_validation_results.jsonl
2026-01-30 08:26:53,781 - testing - INFO - number of test answers longer than 20 words: 0
2026-01-30 08:26:53,782 - testing - INFO - number correct = 111 out of 129
2026-01-30 08:26:53,783 - testing - INFO - percentage correct = 86.04651162790698 %
2026-01-30 08:26:53,803 - testing - INFO - number any word matches = 111 out of 129
2026-01-30 08:26:53,804 - testing - INFO - percentage any word matches = 86.04651162790698 %
2026-01-30 08:26:53,804 - testing - INFO - number of rows with cosine distance less than 0.1: 111
2026-01-30 08:26:53,805 - testing - INFO - percentage of total: 86.04651162790698 %
2026-01-30 08:26:53,806 - testing - INFO - number of rows with cosine distance less than 0.1 and any ma

In [2]:
#count the number of rows in the data frame that have an error in the created commands column
def count_errors_in_df(the_dataframe):
    the_dataframe['created_commands_error'] = the_dataframe['created_commands'].apply(
        lambda x: "# There was an error generating commands for this pre-training sample." in x)
    error_count = the_dataframe['created_commands_error'].sum()
    print("Number of rows with errors in created_commands:", error_count)

In [3]:
#find the row with id = "56d4eb762ccc5a1400d8334f"
dataframe = pipeline.datasetPreProcessor.dataset
dataframe[dataframe[
              "id"] == "57338a51d058e614000b5cf0"]

,id,title,context,question,declarative_statement,answer,declarative_statement_formatted,question_formatted,answer_formatted,question_category,statement_category,is_pretraining,created_commands,created_commands_error
86,57338a51d058e614000b5cf0,University_of_Notre_Dame,Holy Cross Father John Francis O'Hara was elec...,Which person became vice-president of Notre Da...,Father John Francis O'Hara became the Vice-Pre...,Father John Francis O'Hara,Father John_Francis OHara become the Vice - Pr...,? which person become vice - president of Notr...,Father John_Francis OHara,Subject-Verb-Indirect Object-Direct Object,Subject-Verb-Object-Complement,True,"[#id: 57338a51d058e614000b5cf0, Father John_Fr...",False


In [6]:
commands = "[#id: 57338a51d058e614000b5cf0, Father John_Francis OHara become the Vice - President of, the University_of_Notre_Dame in 1933, \n, ? which person become vice - president of Notre_Dame, in 1933, .sctx in 1933, .pg 1933, .sctx ? which person become vice - president of Notre_Dame, .wg become, .ggp, .ph Father John_Francis OHara become the Vice - President of, .drop_goal, .wg Father John_Francis OHara, .rw, \n]"

for command in commands.split(","):
    print(command.strip())



[#id: 57338a51d058e614000b5cf0
Father John_Francis OHara become the Vice - President of
the University_of_Notre_Dame in 1933

? which person become vice - president of Notre_Dame
in 1933
.sctx in 1933
.pg 1933
.sctx ? which person become vice - president of Notre_Dame
.wg become
.ggp
.ph Father John_Francis OHara become the Vice - President of
.drop_goal
.wg Father John_Francis OHara
.rw
]


In [7]:
import pandas as pd

fp = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/sandbox_v2/data_sets/nyc_squad_with_pretraining_commands.jsonl"

df = pd.read_json(fp, lines=True)
count_errors_in_df(df)
df


Number of rows with errors in created_commands: 3


,id,title,context,question,declarative_statement,answer,declarative_statement_formatted,question_formatted,answer_formatted,question_category,statement_category,is_pretraining,created_commands,created_commands_error
0,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ is in front of the N...,a copper statue of Christ,a copper statue of Christ be in front of the N...,? what be in front of the Notre_Dame Main Buil...,a copper statue of Christ,Subject-Verb-Object,Subject-Verb-Object,True,"[#id: 5733be284776f4190066117f, a copper statu...",False
1,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,The Basilica of the Sacred Heart at Notre Dame...,the Main Building,the Basilica of the Sacred Heart at Notre_Dame...,? the Basilica of the sacred heart at Notre_Da...,the Main_Building,Subject-Verb-Object-Complement,Subject-Verb-Object,True,"[#id: 5733be284776f41900661180, the Basilica o...",False
2,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,The Grotto at Notre Dame is a Marian place of ...,a Marian place of prayer and reflection,the Grotto at Notre_Dame be a marian place of ...,? what be the Grotto at Notre_Dame,a marian place of prayer and reflection,Subject-Verb-Object,Subject-Verb-Object,False,[# This is a non-pretraining sample. No comman...,False
3,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary sits on top...,a golden statue of the Virgin Mary,a golden statue of the Virgin_Mary sit on top ...,? what sit on top of the Main_Building at Notr...,a golden statue of the Virgin_Mary,Subject-Verb-Object,Subject-Verb-Object,True,"[#id: 5733be284776f4190066117e, a golden statu...",False
4,5733bf84d058e614000b61c0,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",What is the daily student paper at Notre Dame ...,The daily student paper at Notre Dame is calle...,The Observer,the daily student paper at Notre_Dame be call ...,? what be the daily student paper at Notre_Dam...,the Observer,Subject-Verb-Object,Subject-Verb-Object,False,[# This is a non-pretraining sample. No comman...,False
5,5733bf84d058e614000b61bd,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",How many student news papers are found at Notr...,There are three student newspapers at Notre Dame.,three,there be three student newspaper at Notre_Dame,? how many student news paper be find at Notre...,three,Subject-Verb-Object,Subject-Verb-Object,True,"[#id: 5733bf84d058e614000b61bd, there be three...",False
6,5733bf84d058e614000b61c1,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",In what year did the student paper Common Sens...,the student paper Common Sense began publicati...,1987,the student paper Common Sense begin publicati...,? in what_year do the student paper Common Sen...,1987,Subject-Verb-Object,Subject-Verb-Object,False,[# This is a non-pretraining sample. No comman...,False
7,5733bed24776f4190066118a,University_of_Notre_Dame,The university is the major seat of the Congre...,What is the oldest structure at Notre Dame?,the oldest structure at Notre Dame is Old College,Old College,the old structure at Notre_Dame be Old_College,? what be the old structure at Notre_Dame,Old_College,Subject-Verb-Object,Subject-Verb-Object,False,[# This is a non-pretraining sample. No comman...,False
8,5733bed24776f4190066118b,University_of_Notre_Dame,The university is the major seat of the Congre...,What individuals live at Fatima House at Notre...,The individuals who live at Fatima House at No...,Retired priests and brothers,the individual who live at Fatima

In [3]:
df[df['id'] == "5733a6424776f41900660f50"]

,id,title,context,question,declarative_statement,answer,declarative_statement_formatted,question_formatted,answer_formatted,question_category,statement_category,is_pretraining,created_commands
14,5733a6424776f41900660f50,University_of_Notre_Dame,The College of Engineering was established in ...,How many departments are within the Stinson-Re...,the number of departments within the Stinson-R...,five,the number of department within the Stinson - ...,? how many department be within the Stinson - ...,five,Subject-Verb-Object,Subject-Verb-Object,False,[# This is a non-pretraining sample. No comman...


In [ ]:
pipeline.run_pre_training()
pipeline.run_pre_training_evaluation_testing()
pipeline.run_evaluate_pre_training_results()
pipeline.run_training()
pipeline.run_testing()
pipeline.run_evaluate_training_results()

2026-01-09 13:25:21,261 - pipeline - INFO - Starting pre-training...
2026-01-09 13:25:21,287 - dataset_processing - INFO - Wrote /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/15/pre_training/nyc_squad_pretraining_commands.txt
2026-01-09 13:25:21,289 - dataset_processing - INFO - Number of reward lines: 467
2026-01-09 13:25:21,289 - dataset_processing - INFO - Number of commands: 8080
2026-01-09 13:25:21,292 - training - INFO - copied: /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/15/pre_training/nyc_squad_pretraining_commands.txt to: /Users/chris/PycharmProjects/Training-and-evaluating-cognitive-language-models/docker/shared_data/pre_training
2026-01-09 13:25:21,297 - training - INFO - Running Docker command from directory: /Users/chris/PycharmProjects/Training-and-evaluati